### Convert csv to images for data augmentation purposes

In [4]:
import numpy as np
import os
import pandas as pd
from scipy.misc import imsave

TRAIN_PATH = "data/training"
VALID_PATH = "data/validation"
DATASET_PATHS = (TRAIN_PATH, VALID_PATH)
VALID_PROP = 0.10
IMG_SIZE = 48

imgs = pd.read_csv("data/train_data.csv", header=None).as_matrix()
targets = pd.read_csv("data/train_target.csv", header=None)[0]

for dataset_path in DATASET_PATHS:
    if not os.path.exists(dataset_path):
        os.mkdir(dataset_path)

    for target in targets.unique():
        path_target = os.path.join(dataset_path, str(target))
        if not os.path.exists(path_target):
            os.mkdir(os.path.join(path_target))

for i in range(len(targets)):
    img = imgs[i,:].reshape((IMG_SIZE, IMG_SIZE))
    target = targets[i]
    fname = str(i)+".png"

    if i < VALID_PROP * len(targets):
        path = os.path.join(VALID_PATH, str(target))
    else:
        path = os.path.join(TRAIN_PATH, str(target))

    imsave(os.path.join(path, fname), img)

### Build CNN models

In [5]:
from keras.applications.resnet50 import identity_block, conv_block
from keras.layers import Activation, AveragePooling2D, BatchNormalization, concatenate, Conv2D
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, Input, MaxPooling2D, ZeroPadding2D
from keras.models import Model
from keras.optimizers import Adam

class CNN:

    model = None

    def __init__(self, lr=0.001, decay=0):
        opt = Adam(lr=lr, decay=decay)
        self.model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])

    def Conv2D_bn(self, x, nb_filter, filter_size, strides=(1, 1), padding="same"):
        x = Conv2D(nb_filter, (filter_size, filter_size), strides=strides, padding=padding)(x)
        x = BatchNormalization()(x)
        return Activation("relu")(x)


class VGG(CNN):
    def __init__(self, outputs, input_shape, lr=0.001, decay=0, dropout=0):
        img_input = Input(shape=input_shape)
        x = self.vgg_block(img_input, 2, 64)
        x = self.vgg_block(x, 2, 128)
        x = self.vgg_block(x, 3, 256)
        x = self.vgg_block(x, 3, 512)
        x = Flatten()(x)
        x = self.Dense_bn(x, 1024)
        x = Dropout(dropout)(x)
        x = self.Dense_bn(x, 1024)
        x = Dropout(dropout)(x)
        predictions = Dense(outputs, activation="softmax")(x)
        self.model = Model(inputs=img_input, outputs=predictions)
        super().__init__(lr, decay)

    def vgg_block(self, x, nb_conv, nb_filters, filter_size=3):
        for i in range(nb_conv):
            x = self.Conv2D_bn(x, nb_filters, filter_size)
        return MaxPooling2D((2, 2), strides=(2, 2))(x)
            
    def Dense_bn(self, x, units):
        x = Dense(units)(x)
        x = BatchNormalization()(x)
        return Activation("relu")(x)


class Inception_FCN(CNN):
    def __init__(self, outputs, input_shape, lr=0.001, decay=0, dropout=0):
        img_input = Input(shape=input_shape)
        x = self.inception_block(img_input)
        x = self.inception_block(x)
        x = self.inception_block(x)
        x = self.inception_block(x)
        x = Dropout(dropout)(x)
        x = Conv2D(outputs, (3, 3), padding="same")(x)
        x = GlobalAveragePooling2D()(x)
        predictions = Activation("softmax")(x)
        self.model = Model(inputs=img_input, outputs=predictions)
        super().__init__(lr, decay)

    def inception_block(self, x):
        branch1x1 = self.Conv2D_bn(x, 64, 1, strides=(2, 2))

        branch5x5 = self.Conv2D_bn(x, 48, 1)
        branch5x5 = self.Conv2D_bn(branch5x5, 64, 5, strides=(2, 2))

        branch3x3dbl = self.Conv2D_bn(x, 64, 1)
        branch3x3dbl = self.Conv2D_bn(branch3x3dbl, 96, 3)
        branch3x3dbl = self.Conv2D_bn(branch3x3dbl, 96, 3, strides=(2, 2))

        branch_pool = AveragePooling2D((3, 3), strides=(2, 2), padding="same")(x)
        branch_pool = self.Conv2D_bn(branch_pool, 64, 1)
        return concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool], axis=-1)


class ResNet(CNN):
    def __init__(self, outputs, input_shape, lr=0.001, decay=0, dropout=0):
        img_input = Input(shape=input_shape)

        x = conv_block(img_input, 3, [32, 32, 128], stage=1, block="a", strides=(1, 1))
        x = identity_block(x, 3, [32, 32, 128], stage=1, block="b")
        x = identity_block(x, 3, [32, 32, 128], stage=1, block="c")

        x = conv_block(x, 3, [64, 64, 256], stage=2, block="a")
        x = identity_block(x, 3, [64, 64, 256], stage=2, block="b")
        x = identity_block(x, 3, [64, 64, 256], stage=2, block="c")

        x = conv_block(x, 3, [128, 128, 512], stage=3, block="a")
        x = identity_block(x, 3, [128, 128, 512], stage=3, block="b")
        x = identity_block(x, 3, [128, 128, 512], stage=3, block="c")

        x = conv_block(x, 3, [256, 256, 1024], stage=4, block="a")
        x = identity_block(x, 3, [256, 256, 1024], stage=4, block="b")
        x = identity_block(x, 3, [256, 256, 1024], stage=4, block="c")

        x = GlobalAveragePooling2D(name="global_avg_pool")(x)
        predictions = Dense(outputs, activation="softmax")(x)

        self.model = Model(inputs=img_input, outputs=predictions)
        super().__init__(lr, decay)

Using TensorFlow backend.


### Train 3 models for ensemble

In [7]:
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

BATCH_SIZE = 128
EPOCHS = 120
OUTPUTS = 3

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    horizontal_flip=True,
    width_shift_range=0.10,
    height_shift_range=0.10,
    zoom_range=0.10)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    "data/training",
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True)

validation_generator = test_datagen.flow_from_directory(
    "data/validation",
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False)

if __name__ == "__main__":
    vgg = VGG(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0.001, dropout=0.5, decay=0.0001)
    inception = Inception_FCN(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0.001, dropout=0.4, decay=0.001)
    resnet = ResNet(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0.001, dropout=0.0, decay=0.001)
    cnns = [vgg, inception, resnet]
    for cnn in cnns:
        model_name = type(cnn).__name__
        print("Training {}".format(model_name))
        callbacks = [
            TensorBoard(), 
            ModelCheckpoint("results/{}.h5".format(model_name), monitor="val_acc", save_best_only=True, save_weights_only=True),
            EarlyStopping(monitor="val_acc", patience=20)
        ]
        cnn.model.fit_generator(
            train_generator,
            steps_per_epoch=train_generator.n // BATCH_SIZE + 1,
            epochs=EPOCHS,
            validation_data=validation_generator,
            validation_steps=validation_generator.n // BATCH_SIZE + 1,
            callbacks=callbacks)

Found 14557 images belonging to 3 classes.
Found 1618 images belonging to 3 classes.
Training VGG
Epoch 1/1
114/114 [==============================] - 26s - loss: 1.2578 - acc: 0.4338 - val_loss: 1.1940 - val_acc: 0.2960

### Check ensemble validation accuracy and produce outputs

In [9]:
TEST_FILE = "data/test_data.csv"

validation_generator = test_datagen.flow_from_directory(
    "data/valid",
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=1,
    class_mode="categorical",
    shuffle=False)

def preprocess(img):
    img = img.reshape(IMG_SIZE, IMG_SIZE).astype("float32")
    img /= 255
    img = np.expand_dims(img, axis=0)
    return np.expand_dims(img, axis=-1)

def predict_ensemble(models, img):
    predictions = np.zeros((len(models), OUTPUTS))
    for i in range(len(models)):
        predictions[i] = models[i].predict(img)
    return np.argmax(np.mean(predictions, axis=0)).squeeze()
    
if __name__ == "__main__":
    test_data = pd.read_csv(TEST_FILE, header=None).as_matrix()

    vgg = VGG(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0, dropout=0, decay=0)
    vgg.model.load_weights("results/VGG.h5")
    inception = Inception_FCN(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0, dropout=0, decay=0)
    inception.model.load_weights("results/Inception_FCN.h5")
    resnet = ResNet(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0, dropout=0, decay=0)
    resnet.model.load_weights("results/ResNet.h5")
    models = [vgg.model, inception.model, resnet.model]

    predictions = np.zeros(validation_generator.n)
    targets = np.zeros(validation_generator.n)
    for i, (img, target) in enumerate(validation_generator):
        if i >= validation_generator.n:
            break
        predictions[i] = predict_ensemble(models, img)
        targets[i] = np.argmax(target, axis=1)
    print("Accuracy: {}".format(np.mean(targets == predictions)))

    with open("results/predictions.txt", "w+") as f:
        f.write("Id,Category\n")
        for i in range(len(test_data)):
            img = preprocess(test_data[i, :])
            prediction = predict_ensemble(models, img)
            f.write("{},{}\n".format(i, prediction))

Found 1613 images belonging to 3 classes.
Accuracy: 0.4407935523868568
